# TP -- Pracical work

### Table of Contents

* [Introduction](#introduction)
* [1. Analysing variability, capacity factors](#1.variability)
    * [Q1.1. Load factor (production/consumption)](#1.1.Question)
    * [Q1.2. Most simple dimensioning rules -- Lower and upper bound](#1.2.Question)
    * [Q1.3. Dimensioning -- Adding a storage](#1.3.Question)
* [2. Economic analysis, computing LCOE, system LCOE](#2.LCOE)
    * [Q2.1. LCOE of renewable + Flex system](#2.1.Question)
* [3. Optimisation of operation](#3.Operation)
    * [Q3.1. Optimisation results and Lagrange multipliers](#3.1.Question)
    * [Q3.2. Storage optimisation](#3.2.Question)
* [4. Optimisation of planning](#4.Planning)
    * [Q4.1. Optimisation results and Lagrange multipliers](#4.1.Question)



## Introduction <a class="anchor" id="introduction"></a>

Question asked here can generally be answered by copy-past of code given in folder "BasicFunctionalities" + small modifications/additions of yoru own.


In [ ]:
import os
if os.path.basename(os.getcwd())=="SujetsDAnalyses":
    os.chdir('..') ## to work at project root  like in any IDE

InputFolder='Data/input/'

#region importation of modules
import numpy as np
import pandas as pd
import seaborn as sns
import csv
import datetime
import copy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn import linear_model
from functions.f_operationModels import *
from functions.f_optimization import *
from functions.f_graphicalTools import *
import sys

#endregion

#region Solver and data location definition

InputFolder='Data/input/'

if sys.platform != 'win32':
    myhost = os.uname()[1]
else : myhost = ""
if (myhost=="jupyter-sop"):
    ## for https://jupyter-sop.mines-paristech.fr/ users, you need to
    #  (1) run the following to loanch the license server
    if (os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmgrd -c /opt/mosek/9.2/tools/platform/linux64x86/bin/mosek.lic -l lmgrd.log")==0):
        os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmutil lmstat -c 27007@127.0.0.1 -a")
    #  (2) definition of license
    os.environ["MOSEKLM_LICENSE_FILE"] = '@jupyter-sop'

BaseSolverPath='/Users/robin.girard/Documents/Code/Packages/solvers/ampl_macosx64' ### change this to the folder with knitro ampl ...
## in order to obtain more solver see see https://ampl.com/products/solvers/open-source/
## for eduction this site provides also several professional solvers, that are more efficient than e.g. cbc
sys.path.append(BaseSolverPath)
solvers= ['gurobi','knitro','cbc'] # try 'glpk', 'cplex'
solverpath= {}
for solver in solvers : solverpath[solver]=BaseSolverPath+'/'+solver
solver= 'mosek' ## no need for solverpath with mosek.
#endregion


## 1. Analysing variability, capacity factors  <a class="anchor" id="1.variability"></a>
To answer questions here, you only need to understand thermal sensitivity manipulations explained in file BasicFunctionalities/input-Consumption.ipynb and
and know how to extract Availability data here : BasicFunctionalities/input-Availability.ipynb. You have to know the definition of capacity factor,
and the difference between installed power and energy produced. You can find the corresponding definitions in the first two sections of
the post [here](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).

### Q1.1. Load factor (production/consumption)  <a class="anchor" id="1.1.Question"></a>

**Load factor of consumption**

From the available data, compute load factor of consumption mean($C_t$)/max($C_t$) [%]
for
 - year 2016 for different countries available,
 - years 2013-2017 and for various thermal sensitivity (ranging from zero to 2.5 GW/°C) in the case of France.

In [ ]:
# Put your answer here


**Load factor of Wind Power and PV production**

Compute load factor of renewable production (PV/Wind) for different years/countries that are available (see input-Availability.ipynb).

In [ ]:
# Put your answer here

**"Load factor" of nuclear availability**

Compute the "load factor" of nuclear availability for year 2007 to 2016.

In [ ]:
# Put your answer here


### Q1.2. Most simple dimensioning rules -- Lower and upper bound  <a class="anchor" id="1.2.Question"></a>

 **a -- Lower bound**

How much power of XXX power would you need to install in LLL to have an **annual energy** produced equal to the annual energy consummed in year YYY ?
 Answer this question for XXX = {Wind, PV, Nuke}, LLL =available country and YYY available year.

In [14]:
# Put your answer here


TECHNOLOGIES  HydroReservoir  HydroRiver   OldNuke  Solar  Thermal  \
TIMESTAMP                                                            
1                        1.0    0.446046  0.928560    0.0      1.0   
2                        1.0    0.434063  0.928560    0.0      1.0   
3                        1.0    0.388382  0.928560    0.0      1.0   
4                        1.0    0.347019  0.928560    0.0      1.0   
5                        1.0    0.342092  0.928560    0.0      1.0   
...                      ...         ...       ...    ...      ...   
8756                     1.0    0.702555  0.898972    0.0      1.0   
8757                     1.0    0.559915  0.898972    0.0      1.0   
8758                     1.0    0.477251  0.898972    0.0      1.0   
8759                     1.0    0.435706  0.898972    0.0      1.0   
8760                     1.0    0.494647  0.898972    0.0      1.0   

TECHNOLOGIES  WindOnShore  
TIMESTAMP                  
1                0.583934  
2    

 **b -- Upper bound**

How much power of XXX power would you need to install in LLL to have an **hourly energy** produced equal to the hourly energy consummed in year YYY ?
 Answer this question for XXX = {Wind, Nuke}, LLL =available country and YYY available year.
 Assuming that for all hours of the year the excess of production is lost, what would be the capacity factor of productions in this cases

In [ ]:
# Put your answer here


 **c -- Sensitivity of the result to thermal sensitivity of consumption**

 For year 2016 and in France, analyse the sensitivity of the preceding results (upper bound and lower bound) on the thermal sensitivity of consumption

In [ ]:
# Put your answer here


### Q1.3 Dimensioning -- Adding a storage <a class="anchor" id="1.3.Question"></a>
These questions are much more difficult than the preceding ones. It is possible that no student will manage to do it during the practical work session.

**a - Dimensioning a system with Renewable + "infinite storage"**

We now assume we have a theoretical storage with
 - infinite energy capacity (can store for hours without being full start with an initial capacity to discuss)
 - a maximum production power (of the storage) that will be equal to the maximum consumption power (of the storage)
 - These maximum power are supposed ajusted so that $C_t\leq P_t+S_t$ for all hours $t$ of the considered year
 (With $C$ the consumption,$P$ the wind power production, $S$ the storage production. $S$ can be negative when it is a consumption)
 - an efficiency $\eta=70%$



Try to compute :
 - installed power of wind and storage,
 - losses of wind energy through the storage
 - losses of wind energy that cannot be stored of consumed directly.

In [ ]:
# Put your answer here


**b -adding french hydro power**

 In the preceding system, you can add french hydro production (the data can be found in Eco2mix section of input-Availability.ipynb)

In [ ]:
# Put your answer here

**c -sensitibvity to thermal sensitivity**

 What is the sensitivity of the preceding results to thermal sensitivity of consumption

In [ ]:
# Put your answer here

** d -Adding PV **
In the preceding system, assume now you can mix PV and wind. What can you gain ?

In [ ]:
# Put your answer here


If you finish all this before the end of the first session, of if you wish to dig a little bit more, you can read [this article](https://jancovici.com/transition-energetique/renouvelables/100-renouvelable-pour-pas-plus-cher-fastoche/)
and discuss the dimensioning choice for the storage that is proposed there. Otherwise, or after that, you can switch to question 2.

## 2. Economic analysis, computing LCOE, system LCOE <a class="anchor" id="2.LCOE"></a>

### Q2.1. LCOE of renewable + Flex system <a class="anchor" id="2.1.Question"></a>
Read and understand jupyter notebook input_Economic.ipynb.
Try to imagine systems that are feasible and compute the corresponding system LCOE in the two following cases:
- Nuke + CCG + hydro
- Renewable + CCG + hydro
You will need to set the installed power and produced energy for each of these production mean. Try to get inspiration from preceding section.
There are no "right answer", the idea here is to have rough estimates that will be improved later.
Try adding significant amount of electrolyser/fuel cells to replace CCG. How much capacity would you install and see how it changes the results ?
Try changing the thermal sensitivity of consumption.
You can also analyse the effect of discount rate, or carbon tax, or cost of gaz (by assumign e.g. it is a very expensive biogaz).

In [ ]:
# Put your answer here

## 3. Optimisation of operation <a class="anchor" id="3.Operation"></a>

### Q3.1. Optimisation results and Lagrange multipliers <a class="anchor" id="3.1.Question"></a>
Read and understand jupyter notebook optim-Operation.ipynb. Can you explain the value of all lagrange multiplier in a simple case with two production means ?
With one area and then with two. How is it possible to generate negative lagrange multipliers associated to the constraint production == consumption ? Propose a simple case where they appear and explain the corresponding values.


In [ ]:
# Put your answer here

### Q3.2. Storage optimisation <a class="anchor" id="3.2.Question"></a>
Read and understand jupyter notebook optim-storage.ipynb. What is the maximal price for the storage system to be profitable today ?
Apply an homothetic transformation to increase the variance of the prices but without changing the mean (and by keeping positive prices).
How does it change the preceding results ?

In [ ]:
# Put your answer here


## 4. Optimisation of planning <a class="anchor" id="4.Planning"></a>

### Q4.1. Optimisation results and Lagrange multipliers <a class="anchor" id="4.1.Question"></a>

**a - System LCOE**

Read and understand jupyter notebook optim-storage.ipynb. Try using it to adapt your answer to Q2.1.

In [ ]:
# Put your answer here

**a - Loss of load expectation**

Today the expected loss of load is of 3 hours in average per year, try to find how to set the variable cost of "curtailment"
(consumption that is cut, the cost is initially set to 3000 €/MWh) to have the value of this expected number at 3.1.
Try to find in the CRE or RTE or ENEDIS website the "official value" in France.

In [ ]:
# Put your answer here